In [11]:
import os 
import pandas as pd
import numpy as np
import h5py
import json
from numpy import nan
import requests
import os.path
from sklearn import preprocessing

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Load Data

In [12]:
def get_test_data():
    df = pd.read_hdf('./checkpoints/data_test.h5','table')
    return df.values

In [13]:
def normalize_data(float_data):
    mean = float_data.mean(axis=0)
    float_data -= mean
    std = float_data.std(axis=0)
    float_data /= std
    return float_data

In [14]:
data = get_test_data()
data = normalize_data(data)

### Create Train Test Split

In [15]:
# Coppied from Deep Learning with Python by Francois Chollet
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    """ Generator yielding timeseries samples and their targets
    
    Arguments:
        data {[type]} -- The original array of floating-point data,
        lookback {[type]} -- How many timesteps back the input data should go.
        delay {[type]} -- How many timesteps in the future the target should be.
        min_index {[type]} -- Indices in the data array that delimit which timesteps to draw from.
        max_index {[type]} -- Indices in the data array that delimit which timesteps to draw from.
    
    Keyword Arguments:
        shuffle {bool} -- Whether to shuffle the samples or draw them in chronological order. (default: {False})
        batch_size {int} -- The number of samples per batch. (default: {128})
        step {int} -- The period, in timesteps, at which you sample data. (default: {6})
    """

    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [16]:
def get_train_test_val(data):
    train_index = int(len(data) * (.5))
    test_index = train_index + int(len(data) * (.25))
    
    lookback = 10080
    step = 1
    delay = 1440
    
    train_gen = generator(data,
                         lookback=lookback,
                         delay=delay,
                         min_index=0,
                         max_index=train_index,
                         step=step
                         )
    
    test_gen = generator(data,
                         lookback=lookback,
                         delay=delay,
                         min_index=train_index + 1,
                         max_index=test_index,
                         step=step
                         )
    
    val_gen = generator(data,
                         lookback=lookback,
                         delay=delay,
                         min_index=test_index + 1,
                         max_index=len(data),
                         step=step
                         )
    
    val_steps = (len(data) - (test_index +1) - lookback) // 128
    
    test_steps = (test_index - (train_index +1) - lookback) // 128
    
    return train_gen, test_gen, val_gen

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

Using TensorFlow backend.


In [26]:
def get_model(float_data):
    model = Sequential()
    model.add(Flatten(input_shape=(lookback // step, float_data.shape[-1])))
    model.add(Dense(32), activation='relu')
    model.add(Dense(len(float_data.shape[-1])))

In [ ]:
def train_model():
    float_data = get_test_data()
    